In [1]:
import sys
sys.path.append("../")

In [2]:
# from __future__ import division
import tensorflow as tf
import numpy as np
from collections import deque
import random
import gym
from gym import wrappers
from gym.envs.classic_control.pendulum import angle_normalize, PendulumEnv
from core import *
from utils_latentPolicy_sac_lstm_zt_zt1_ensemble import *
import os
import tensorflow_probability as tfp
import multiprocessing as mp
import os
import d4rl
import json
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

slim = tf.contrib.slim
rnn = tf.contrib.rnn
tfd = tfp.distributions
config=tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth = True

2022-09-09 14:33:43.355142: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



No module named 'flow'
/home/gaoqitong/anaconda3/envs/ope_py37/lib/python3.8/site-packages/glfw/__init__.py:906: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: Dec  1 2021 18:33:43


In [3]:
def evaluate(args):
    ope_path, target_policy_path = args
    
    try:
        
#         result_df_path = "./d4rl_results_ensemble_d4rlOnly_medium/" + target_policy_path.split("/")[-1].split(".")[0]+".txt"
        
#         if os.path.exists(result_df_path):
        
#             result_df = pd.read_csv(result_df_path, delimiter="\t", header=None)
#             if ope_path.split("/")[-2].split("_")[-9:-1] in result_df.loc[:,:8].values:
#                 print ("Existed "+ope_path+" "+result_df_path)
#                 return

        LR = 0.0003
        GAMMA = .995
        BUFFER_SIZE_SAC = 2*10**6
        MINIBATCH_SIZE_SAC = 256
        MINIBATCH_SIZE_OPE = 64
        RANDOM_SEED = 2599
        MAX_EPISODES = 1000
        MAX_EPISODE_LEN = 1000
        NUM_OPE_MODELS = 3
        CODE_SIZE = 16
        EXPLORATION = .3
        REPEAT = 1
        BUFFER_SIZE_OPE = 2000
        beta = 1.


        OPE_LR = 0.001
        OPE_DS = 1000
        OPE_DR = 0.98

        BEST_MAE = 9999.


        network_params = {
        'hidden_sizes':[256, 256],
        'activation':'relu',
        'policy':mlp_gaussian_policy
        }

        rl_params = {
            'env_name':'halfcheetah-expert-v2',

            # control params
            'seed': RANDOM_SEED,
            'epochs': MAX_EPISODES,
            'actor_critic':mlp_actor_critic,
            'steps_per_epoch': MAX_EPISODE_LEN,
            'replay_size': BUFFER_SIZE_SAC,
            'batch_size': MINIBATCH_SIZE_SAC,
            'start_epis': 0,
            'max_ep_len': MAX_EPISODE_LEN,
            'save_freq': 10,
            'render': False,

            # rl params
            'gamma': GAMMA,
            'polyak': 0.995,
            'lr': LR,
            'grad_clip_val':None,

            # entropy params
            'alpha': 'auto',
            'target_entropy':'auto' # fixed or auto define with -act_dim
        }

        file_appendix = (
            "OPE_SAC_latentPolicy_lstm_zt_zt1_ensemble_d4rlOnly_" + rl_params['env_name'] + "_" + str(MAX_EPISODES)
            + "epi_repeat"+ str(REPEAT) + "_" + str(LR) + "_"
            + str(OPE_LR) + "_"
            + str(OPE_DS) + "_"
            + str(OPE_DR) + "_"
            + str(CODE_SIZE) + "_"
            + str(beta) + "_"
            + str(RANDOM_SEED)
        )

        env = gym.make(rl_params['env_name'])
        np.random.seed(RANDOM_SEED)
        tf.set_random_seed(RANDOM_SEED)
        env.seed(RANDOM_SEED)

        env_state_dim = env.observation_space.shape[0]
        # state_dim = CODE_SIZE
        env_action_dim = env.action_space.shape[0]
        env_action_bound = env.action_space.high
        env_state_bound = None
        # Ensure action bound is symmetric
    #     assert (env.action_space.high == -env.action_space.low)

        graph_ope_models = tf.Graph()

        graph_ac = tf.Graph()

        with tf.Session(config=config, graph=graph_ope_models) as sess_ope_models:

            with graph_ope_models.as_default():

                ope_model = OPE_Model(
                    NUM_OPE_MODELS, graph_ope_models, sess_ope_models, OPE_LR, OPE_DS, OPE_DR, CODE_SIZE,
                    env_state_dim, env_state_bound, env_action_dim, file_appendix,
                    BUFFER_SIZE_OPE, RANDOM_SEED, MINIBATCH_SIZE_OPE, MAX_EPISODE_LEN, beta,
                    is_training=False
                )

                ope_saver = ope_model.saver

                ope_saver.restore(sess_ope_models, ope_path)


                d4rl_qlearning = d4rl.qlearning_dataset(env)

                obs_mean = d4rl_qlearning['observations'].mean(0).astype(np.float32)
                obs_std = d4rl_qlearning['observations'].std(0).astype(np.float32)

                rew_mean = d4rl_qlearning['rewards'].mean()
                rew_std = d4rl_qlearning['rewards'].std()

                class LearnedEnv(object):
                    def __init__(self, model):

            #             super(LearnedPendulum, self).__init__()
                        self.model = model

                    def reset(self):
                        s0 = self.model.init_z0_s0()
            #                 s0 = np.array([ 0.76898139, -0.63927117,  0.30185718])
            #             s0 = self.model.sess.run(self.model.decoder_state_sample, 
            #                                feed_dict={self.model.decoder_zt_holder:self.model.zt}).reshape(-1)

                        self.obs = s0
                        return s0

                    def step(self, u):
                        new_obs, reward = self.model.get_zt1_s2_r(np.reshape(u, (1, env_action_dim)))
                        self.obs = new_obs
                        self.model.update_zt()

                        return new_obs, reward, False, {}

                learned_env = LearnedEnv(ope_model)

                np.random.seed(RANDOM_SEED)
                tf.set_random_seed(RANDOM_SEED)

                ep_rewards = []
                policy = D4RL_Policy(target_policy_path)
                for i in range(50):

                    terminal = 0

                    s = learned_env.reset()
                    s = s.reshape(env_state_dim)*obs_std + obs_mean
                    ep_reward = 0

                    for j in range(MAX_EPISODE_LEN):

                        if j % REPEAT == 0:
                            a, _ = policy.act(np.reshape(s, (env_state_dim,)), np.zeros((env_action_dim,)))
                        s2, r, terminal, info = learned_env.step(a)
                        r = r*rew_std + rew_mean
                        s2 = s2.reshape(env_state_dim)*obs_std + obs_mean

                        ep_reward += r*(GAMMA**j)

                        s = s2

                        if terminal or j == MAX_EPISODE_LEN-1:
                            ep_rewards += [ep_reward]

                            break

                with open("./d4rl_results_ensemble_d4rlOnly_expert_gamma_995_1234/"+target_policy_path.split("/")[-1].split(".")[0]+".txt", "a") as myfile:
                    myfile.write("\t".join(ope_path.split("/")[-2].split("_")[-9:-1] + [str(np.mean(ep_rewards))]) + "\n")

    except:
        return

In [4]:
with tf.io.gfile.GFile("../../d4rl/deep_ope/d4rl_policies.json", 'r') as f:
    policy_database = json.load(f)
policy_metadatas = [i for i in policy_database if i['task.task_names'][0].find("halfcheetah")!=-1]

OPEs = ["./saved_model/"+i+"/ope_best.ckpt" for i in os.listdir("./saved_model/") if (i.find("OPE_SAC_latentPolicy_lstm_zt_zt1_ensemble_d4rlOnly_halfcheetah-expert-v2_850epi")!=-1 and i.find("1234")!=-1 )]
TARGETs = [p['policy_path'] for p in policy_metadatas]


In [ ]:
pool = mp.Pool(3)
pool.map(evaluate, [(o_path, t_path) for o_path in OPEs for t_path in TARGETs])
pool.close()
pool.join()

/home/gaoqitong/anaconda3/envs/ope_py37/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/gaoqitong/anaconda3/envs/ope_py37/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/gaoqitong/anaconda3/envs/ope_py37/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")





2022-09-09 14:33:46.370040: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3493465000 Hz
2022-09-09 14:33:46.373436: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559cc7a6d1f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-09 14:33:46.373480: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-09-09 14:33:46.376298: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-09 14:33:46.386822: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3493465000 Hz
2022-09-09 14:33:46.389961: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559cc7a6d1f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-09 14:33:46.390009: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host

self.encoder_zt_out_from_st_distself.encoder_zt_out_from_st_dist  tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag", batch_shape=[?], event_shape=[16], dtype=float32)tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag", batch_shape=[?], event_shape=[16], dtype=float32)

self.encoder_zt_out_from_st_sampleself.encoder_zt_out_from_st_sample Tensor("Encoder_zt/Encoder_zt_MultivariateNormalDiag/sample/Encoder_zt_MultivariateNormalDiag_affine_linear_operator/forward/add:0", shape=(?, 16), dtype=float32) 
Tensor("Encoder_zt/Encoder_zt_MultivariateNormalDiag/sample/Encoder_zt_MultivariateNormalDiag_affine_linear_operator/forward/add:0", shape=(?, 16), dtype=float32)self.encoder_zt1_out_from_st1_dist 
self.encoder_zt1_out_from_st1_disttfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag_1", batch_shape=[?], event_shape=[16], dtype=float32) 
tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag_1", ba

2022-09-09 14:33:46.924740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-09-09 14:33:46.924949: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 
2022-09-09 14:33:46.924966: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N 
2022-09-09 14:33:46.925310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 14:33:46.926962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-09-09 14:33:46.927048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 
2022-09-09 14:33:46.927056: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N 
2022-09-09 14:33:46.927368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node

self.encoder_zt_out_from_st_dist tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag", batch_shape=[?], event_shape=[16], dtype=float32)
self.encoder_zt_out_from_st_sample Tensor("Encoder_zt/Encoder_zt_MultivariateNormalDiag/sample/Encoder_zt_MultivariateNormalDiag_affine_linear_operator/forward/add:0", shape=(?, 16), dtype=float32)
self.encoder_zt1_out_from_st1_dist tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag_1", batch_shape=[?], event_shape=[16], dtype=float32)
self.encoder_zt1_out_from_st1_sample Tensor("Encoder_zt/Encoder_zt_MultivariateNormalDiag_1/sample/Encoder_zt_MultivariateNormalDiag_1_affine_linear_operator/forward/add:0", shape=(?, 16), dtype=float32)
self.encoder_zt1_out_from_zt_dist tfp.distributions.MultivariateNormalDiag("Encoder_zt1_1_MultivariateNormalDiag", batch_shape=[?], event_shape=[16], dtype=float32)
self.encoder_zt1_out_from_zt_sample self.encoder_zt1_out_from_zt_dist Tensor("Encoder_zt1_1/Encoder_zt1_1_M

...

load datafile: 100%|██████████| 21/21 [00:09<00:00,  2.28it/s]


self.likelihood_s Tensor("Mean:0", shape=(), dtype=float32)
self.likelihood_r Tensor("Mean_1:0", shape=(), dtype=float32)
self.divergence1 Tensor("Mean_2:0", shape=(), dtype=float32)
self.divergence2 Tensor("Mean_3:0", shape=(), dtype=float32)
self.encoder_decoder_lstm_states_mse Tensor("Mean_5:0", shape=(), dtype=float32)
INFO:tensorflow:Restoring parameters from ./saved_model/OPE_SAC_latentPolicy_lstm_zt_zt1_ensemble_d4rlOnly_halfcheetah-expert-v2_850epi_repeat1_0.0003_0.0003_1000_0.997_16_0.5_1234/ope_best.ckpt


INFO:tensorflow:Restoring parameters from ./saved_model/OPE_SAC_latentPolicy_lstm_zt_zt1_ensemble_d4rlOnly_halfcheetah-expert-v2_850epi_repeat1_0.0003_0.0003_1000_0.997_16_0.5_1234/ope_best.ckpt
load datafile: 100%|██████████| 21/21 [00:03<00:00,  5.84it/s]
/home/gaoqitong/anaconda3/envs/ope_py37/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
2022-09-11 00:40:56.028716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-11 00:40:56.029051: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1665] Found device 0 with properties: 
name: NVIDIA RTX A5000 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:41:00.0
2022-09-11 00:40:56.029088: I tensorflow/stream_executor/platform/default/dso_l

self.encoder_zt_out_from_st_dist tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag", batch_shape=[?], event_shape=[16], dtype=float32)
self.encoder_zt_out_from_st_sample Tensor("Encoder_zt/Encoder_zt_MultivariateNormalDiag/sample/Encoder_zt_MultivariateNormalDiag_affine_linear_operator/forward/add:0", shape=(?, 16), dtype=float32)
self.encoder_zt1_out_from_st1_dist tfp.distributions.MultivariateNormalDiag("Encoder_zt_MultivariateNormalDiag_1", batch_shape=[?], event_shape=[16], dtype=float32)
self.encoder_zt1_out_from_st1_sample Tensor("Encoder_zt/Encoder_zt_MultivariateNormalDiag_1/sample/Encoder_zt_MultivariateNormalDiag_1_affine_linear_operator/forward/add:0", shape=(?, 16), dtype=float32)
self.encoder_zt1_out_from_zt_dist tfp.distributions.MultivariateNormalDiag("Encoder_zt1_1_MultivariateNormalDiag", batch_shape=[?], event_shape=[16], dtype=float32)
self.encoder_zt1_out_from_zt_sample Tensor("Encoder_zt1_1/Encoder_zt1_1_MultivariateNormalDiag/sample/Encod

INFO:tensorflow:Restoring parameters from ./saved_model/OPE_SAC_latentPolicy_lstm_zt_zt1_ensemble_d4rlOnly_halfcheetah-expert-v2_850epi_repeat1_0.0003_0.0003_1000_0.997_16_1.0_1234/ope_best.ckpt
load datafile: 100%|██████████| 21/21 [00:08<00:00,  2.50it/s]
